In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random


2024-01-22 19:56:34.249093: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_network(network_params):
    model = Sequential()
    model.add(Dense(network_params['first_layer_neurons'], activation=network_params['activation'], input_shape=(784,)))
    for i in range(network_params['hidden_layers']):
        model.add(Dense(network_params['hidden_layer_neurons'], activation=network_params['activation']))
    model.add(Dense(10, activation='softmax'))  # 10 sınıf için çıkış katmanı
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def generate_initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        network_params = {
            'first_layer_neurons': random.choice([32, 64, 128]),
            'hidden_layers': random.choice([1, 2, 3]),
            'hidden_layer_neurons': random.choice([32, 64, 128]),
            'activation': random.choice(['relu', 'tanh'])
        }
        population.append(network_params)
    return population

def compute_fitness(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train, epochs=3, verbose=0)  # Eğitim süresini kısa tutmak için 3 epoch
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    return accuracy

def select_parents(population, fitness, num_parents):
    parents = np.argsort(fitness)[-num_parents:]
    return [population[i] for i in parents]

def crossover(parents):
    offspring = []
    for _ in range(len(parents) // 2):
        parent1, parent2 = random.sample(parents, 2)
        child = parent1.copy()
        for key in parent2:
            if random.uniform(0, 1) < 0.5:
                child[key] = parent2[key]
        offspring.append(child)
    return offspring

def mutate(offspring):
    for child in offspring:
        if random.uniform(0, 1) < 0.1:  # %10 mutasyon şansı
            child['first_layer_neurons'] = random.choice([32, 64, 128])
            child['hidden_layers'] = random.choice([1, 2, 3])
            child['hidden_layer_neurons'] = random.choice([32, 64, 128])
            child['activation'] = random.choice(['relu', 'tanh'])
    return offspring


In [3]:
# MNIST veri setini yükleme
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
X_train = X_train.reshape(X_train.shape[0], 784)
X_val = X_val.reshape(X_val.shape[0], 784)
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)

# Genetik algoritma parametreleri
population_size = 10
num_generations = 5
num_parents = 4

# Başlangıç popülasyonunu oluşturma
population = generate_initial_population(population_size)

# Her nesil için döngü
for generation in range(num_generations):
    print(f"Generation {generation + 1}")
    fitness = []
    for i, network_params in enumerate(population):
        model = create_network(network_params)
        accuracy = compute_fitness(model, X_train, y_train, X_val, y_val)
        fitness.append(accuracy)
        print(f"Network {i + 1}: Accuracy {accuracy}")

    parents = select_parents(population, fitness, num_parents)
    offspring_crossover = crossover(parents)
    offspring_mutation = mutate(offspring_crossover)

    population = parents + offspring_mutation


11490434/11490434 [==============================] - 5s 0us/step
Generation 1
Network 1: Accuracy 0.8274999856948853
Network 2: Accuracy 0.8368333578109741
Network 3: Accuracy 0.9196666479110718
Network 4: Accuracy 0.9316666722297668
Network 5: Accuracy 0.9258333444595337
Network 6: Accuracy 0.8895000219345093
Network 7: Accuracy 0.9440000057220459
Network 8: Accuracy 0.8974999785423279
Network 9: Accuracy 0.9318333268165588
Network 10: Accuracy 0.9448333382606506
Generation 2
Network 1: Accuracy 0.9355000257492065
Network 2: Accuracy 0.9568333625793457
Network 3: Accuracy 0.9356666803359985
Network 4: Accuracy 0.9448333382606506
Network 5: Accuracy 0.9455000162124634
Network 6: Accuracy 0.9313333630561829
Generation 3
Network 1: Accuracy 0.9325000047683716
Network 2: Accuracy 0.9351666569709778
Network 3: Accuracy 0.949999988079071
Network 4: Accuracy 0.9363333582878113
Network 5: Accuracy 0.9368333220481873
Network 6: Accuracy 0.940500020980835
Generation 4
Network 1: Accuracy 0.9506